# Preparing a terra.bio sample table from GEO/SRA
TODO

In [15]:
# bioproject <- "PRJNA668299" # Mellor Lab - Spt4
bioproject <- "PRJNA669852"# Churchman Lab - dozens of regulatory factors
genomeName <- "sacCer3"
genome_fasta <- "https://hgdownload.soe.ucsc.edu/goldenPath/sacCer3/bigZips/sacCer3.fa.gz"

In [4]:
# Load needed packages, installing if necessary
required_packages <- c("AnVIL", "xml2", "rentrez", "glue", "tidyverse", "kableExtra",
                      "TxDb.Scerevisiae.UCSC.sacCer3.sgdGene")
install_and_load <- function(packages) {
    if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
        BiocManager::install(setdiff(packages, rownames(installed.packages())))
    }
    lapply(packages, library,  character.only = TRUE)
    NULL
}
x <- install_and_load(required_packages)

# Define functions
# NOTE: use rentrez to avoid using the SRAdb secondary source.
# filetype in "fastq", "run"
create_sample_grid <- function(bioproject,
            semantic_name = "fastq", org = "GCP", retmax = NULL) {

        bpid <- entrez_search(db = "bioproject", term = bioproject)
    if (bpid$count == 0) {
        stop(glue::glue("Bioproject <{bioproject}> not found"))
    }

    sra_ids <- entrez_link(dbfrom = "bioproject", id = bpid$ids, db = "sra")$links$bioproject_sra
    x <- read_xml(entrez_fetch(db = "sra", id = sra_ids, rettype = "xml", retmax = retmax))
    
    runs <- xml_find_all(x, '/EXPERIMENT_PACKAGE_SET/EXPERIMENT_PACKAGE/RUN_SET/RUN')
    run_id <- xml_attr(runs, "accession")
    expref <- xml_find_all(runs, "EXPERIMENT_REF")
    experiment_id <- xml_attr(expref, "accession")
    biosample_id <- xml_attr(expref, "refname")
    # Only reporting first member in pool
    member <- xml_find_first(runs, "Pool/Member")
    sample_title <- xml_attr(member, "sample_title")
    sra_sample_id <- xml_attr(member, "accession")
    sra_File <- xml_find_first(runs, glue("./SRAFiles/SRAFile[@semantic_name=\"{semantic_name}\"]"))
    sra_File_alt <- xml_find_first(sra_File, glue("./Alternatives[@org=\"{org}\"]"))
    filename <- xml_attr(sra_File, "filename")
    sra_url <- xml_attr(sra_File_alt, "url")
    tibble(bioproject, experiment_id, biosample_id, sra_sample_id, run_id, sample_title,  filename, sra_url)
}

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cloud.r-project.org


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Installing package(s) 'TxDb.Scerevisiae.UCSC.sacCer3.sgdGene'

also installing the dependencies ‘SummarizedExperiment’, ‘Rsamtools’, ‘GenomicAlignments’, ‘GenomicRanges’, ‘XVector’, ‘Biostrings’, ‘rtracklayer’, ‘biomaRt’, ‘Biobase’, ‘KEGGREST’, ‘GenomicFeatures’, ‘AnnotationDbi’


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ stringr 1.4.0
✔ tidyr   1.1.4     ✔ forcats 0.5.1
✔ readr   2.1.1     

── Conflicts ───────────────────────

In [35]:
# TODO unwrap create_sample_grid
bpid <- entrez_search(db = "bioproject", term = bioproject)
bp_summary <- entrez_summary(db = "bioproject", id = bpid$ids)
sapply(c("project_acc",
    "project_name",
    "submitter_organization",
    "registration_date"), 
       function(u) data.frame(attribute = u, value = bp_summary[[u]]), USE.NAMES = FALSE) %>% 
 t %>% kable(format = "pipe", caption = "Bioproject Summary Attributes")



Table: Bioproject Summary Attributes

|attribute              |value                                                                                 |
|:----------------------|:-------------------------------------------------------------------------------------|
|project_acc            |PRJNA669852                                                                           |
|project_name           |Dynamics of transcription elongation are finely-tuned by dozens of regulatory factors |
|submitter_organization |Stirling Churchman, Genetics, Harvard Medical School                                  |
|registration_date      |2020/10/19 00:00                                                                      |

In [44]:
result <- create_sample_grid(bioproject)

# TODO Clean up sample_id's if we are going to allow multiple assays
# Infer strain and assay type from sample_title
result %>% 
    separate(sample_title, into = c("sample_id", "assay"), sep = "_", remove = FALSE) %>%
    separate(sample_id, into = "strain", sep = "-", extra = "drop", remove = FALSE) %>%
    filter(assay == "netseq") %>%
    relocate(sample_id) %>%
    arrange(sample_title) %>%
    dplyr::rename(inputFastQ = sra_url, "entity:sample_id" = sample_id) -> sample

sample %>% avtable_import

[1] "sample"

In [34]:
# Copy genome fasta to local bucket
genome_local_fa <- glue(tempdir(), "/", genomeName, ".fa")
download.file(genome_fasta, genome_local_fa)
gs_uri <- glue(avbucket(), "/", genomeName, ".fa")
gsutil_cp(genome_local_fa, gs_uri)

Copying file:///tmp/RtmpWWPBlc/sacCer3.fa [Content-Type=application/octet-stream]...
/ [0/1 files][    0.0 B/  3.6 MiB]   0% Done                                    
/ [1/1 files][  3.6 MiB/  3.6 MiB] 100% Done                                    
Operation completed over 1 objects/3.6 MiB.                                      

In [ ]:
# advertise location of genome in the workspace data table
tibble(`workspace:refFasta` = gs_uri) %>% avtable_import